<a href="https://colab.research.google.com/github/cpython-projects/da_1709/blob/main/lesson_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/cpython-projects/da_1305/refs/heads/main/real_estate_data.csv')
df.head()

,Unnamed: 0,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,kitchen_area,balcony,locality_name,days_exposition
0,0,7312500.0,108.0,2024-05-15,3,2.70,16.0,51.0,8,NaN,False,25.0,NaN,Kyiv,NaN
1,1,1884375.0,40.4,2024-08-14,1,NaN,11.0,18.6,1,NaN,False,11.0,2.0,Brovary,81.0
2,2,2922750.0,56.0,2023-11-06,2,NaN,5.0,34.3,4,NaN,False,8.3,0.0,Kyiv,558.0
3,3,36506250.0,159.0,2024-03-19,3,NaN,14.0,NaN,9,NaN,False,NaN,0.0,Kyiv,424.0
4,4,5625000.0,100.0,2024-06-12,2,3.03,14.0,32.0,13,NaN,False,41.0,NaN,Kyiv,121.0


# Групування даних з `groupby()`

## Що таке `groupby()`

Метод `groupby()` дозволяє згрупувати дані за одним або кількома критеріями для подальшого узагальнення (агрегації).

**Загальна формула:**

```python
df.groupby(ключ)[поля].агрегуюча_функція()
```

## Сценарії використання

In [1]:
# Групування по одному полю — одна метрика


In [2]:
# Групування по одному полю — кілька метрик


In [3]:
# Групування по кількох полях


In [4]:
# Агрування кількох полів — кілька метрик


## `reset_index()`

Після `groupby()` індексом стає групувальне поле (або поля). Щоб зробити з них **звичайні колонки**, потрібно:

```python
.reset_index()
```

Інакше графіки чи подальша робота будуть ускладнені.

## Задача: обчислити:

* середню ціну (`last_price`)
* кількість оголошень (`count`)
* медіанну площу (`total_area`)
  по кожному місту

In [5]:
# Візуалізація: Топ-10 міст за кількістю оголошень


# Агрегації `.agg()` в Pandas

## Навіщо потрібні агрегації?

Агрегація — це зведення (узагальнення) числових значень:

| Приклади запитань                      | Агрегація  |
| -------------------------------------- | ---------- |
| Яка середня площа квартир у Києві?     | `mean()`   |
| Скільки квартир виставлено у Броварах? | `count()`  |
| Яка максимальна ціна у студій?         | `max()`    |
| Яка медіанна кухня у 3-кімнатних?      | `median()` |

---

## Синтаксис `.agg()` — на одному або кількох полях

```python
df.agg({'col1': 'mean', 'col2': 'max'})
```

Або на одній колонці:

```python
df['last_price'].agg(['mean', 'median', 'std'])
```

---

## Комбінування з `groupby()`

```python
df.groupby('locality_name').agg({
    'last_price': ['mean', 'median', 'min', 'max'],
    'total_area': 'mean'
})
```

У результаті — **ієрархічний заголовок колонок (MultiIndex)** → бажано `.reset_index()` та перейменувати колонки.

In [ ]:
summary = df[['last_price', 'total_area', 'kitchen_area']].agg(
    ['mean', 'median', 'std', 'min', 'max', 'count']
).round(1)

summary


,last_price,total_area,kitchen_area
mean,3679621.2,60.3,10.6
median,2615625.0,52.0,9.1
std,6123945.0,35.7,5.9
min,6856.0,12.0,1.3
max,429187500.0,900.0,112.0
count,23699.0,23699.0,21421.0


In [ ]:
# T - transpose

summary = df[['last_price', 'total_area', 'kitchen_area']].agg(
    ['mean', 'median', 'std', 'min', 'max', 'count']
).T.round(1)

summary

,mean,median,std,min,max,count
last_price,3679621.2,2615625.0,6123945.0,6856.0,429187500.0,23699.0
total_area,60.3,52.0,35.7,12.0,900.0,23699.0
kitchen_area,10.6,9.1,5.9,1.3,112.0,21421.0


In [6]:
# Візуалізація: Boxplot — ціна квартир по кількості кімнат


In [7]:
# Візуалізація: Histogram — площа квартир


## Нюанси

| Питання                       | Коментар                                     |
| ----------------------------- | -------------------------------------------- |
| `NaN`                         | Ігноруються у функціях агрегування           |
| `.agg()` після `groupby()`    | Повертає MultiIndex → `.reset_index()`       |
| `std`, `min`, `max`, `median` | Обчислюються лише для числових               |
| Для bool                      | Можна використовувати `mean()` — доля `True` |

---

# Зведені таблиці (`pivot_table`)

## Що таке `pivot_table`

`pivot_table()` — це розширений варіант `groupby`, який дозволяє:

* Зробити зведену таблицю по кількох категоріях
* Розмістити одну категорію по рядках, іншу по стовпцях
* Одразу застосувати агрегуючу функцію
* Аналог Excel PivotTable!

---

## Синтаксис

```python
pd.pivot_table(
    data=df,
    values='метрика',
    index='рядки',
    columns='стовпці',
    aggfunc='функція'
)
```

In [8]:
# Приклад: середня ціна по містах і кількості кімнат


In [9]:
# Візуалізація: Heatmap зведеної таблиці


## Додаткові параметри

| Параметр                     | Значення            |
| ---------------------------- | ------------------- |
| `fill_value`                 | чим заповнити `NaN` |
| `margins=True`               | додати підсумки     |
| `aggfunc=['mean', 'median']` | кілька функцій      |


# Частотні таблиці — `pd.crosstab()`

## Що таке `crosstab`

`pd.crosstab()` — це спосіб підрахунку **кількості випадків** для комбінацій категоріальних змінних.

## Синтаксис

```python
pd.crosstab(index=df['рядки'], columns=df['стовпці'])
```

In [10]:
# Приклад: скільки квартир є студіями або ні по містах


In [11]:
# Приклад: студії × відкритий план


## Нюанси

| Питання                            | Коментар                              |
| ---------------------------------- | ------------------------------------- |
| `NaN` у категоріях                 | не враховуються                       |
| `normalize='index'`                | показує **частки** замість кількостей |
| Можна додати `values=`, `aggfunc=` | як у pivot\_table                     |

---

In [12]:
# Частки типів квартир (кількість кімнат) у кожному місті


In [13]:
# Середня площа квартир по населених пунктах і кількості кімнат



In [14]:
# Загальна ціна (last\_price) по типу квартири (студія, не студія) і наявності відкритого планування


# `resample()` — агрегування за датами

## Що таке `resample`

`resample()` — це аналог `groupby()` **по часу**, який працює з колонкою `datetime`.

Необхідно:

* мати колонку типу `datetime64[ns]`
* встановити її як **індекс**

---

## Синтаксис:

```python
df.resample('період')['метрика'].agg(функція)
```


### Базові часові одиниці

| Частота     | Alias             | Опис             |
| ----------- | ----------------- | ---------------- |
| Day         | `'D'`             | календарний день |
| Hour        | `'H'` або `'h'`   | година           |
| Minute      | `'T'` або `'min'` | хвилина          |
| Second      | `'S'` або `'s'`   | секунда          |
| Millisecond | `'ms'`            | мілісекунда      |
| Microsecond | `'us'`            | мікросекунда     |
| Nanosecond  | `'ns'`            | наносекунда      |

---

### Бізнес-час і бізнес-дні

| Частота                  | Alias   | Опис                                                 |
| ------------------------ | ------- | ---------------------------------------------------- |
| BDay / BusinessDay       | `'B'`   | бізнес-день (пн–пт)                                  |
| CDay / CustomBusinessDay | `'C'`   | кастомний бізнес-день (за користувацьким календарем) |
| BusinessHour             | `'bh'`  | бізнес-година                                        |
| CustomBusinessHour       | `'cbh'` | кастомна бізнес-година                               |

---

### Тижні

| Частота         | Alias    | Опис                                           |
| --------------- | -------- | ---------------------------------------------- |
| Week            | `'W'`    | тиждень, можна якір на день (`W-MON`, `W-FRI`) |
**Приклади**

* `'W'` — тиждень, кінець у неділю
* `'W-MON'` — тиждень з кінцем у понеділок

---

### Місячні частоти

| Частота        | Alias              | Опис                            |
| -------------- | ------------------ | ------------------------------- |
| MonthEnd       | `'M'` або `'ME'`   | кінець календарного місяця      |
| MonthBegin     | `'MS'`             | початок місяця                  |
| BMonthEnd      | `'BM'` або `'BME'` | бізнес-кінець місяця            |
| BMonthBegin    | `'BMS'`            | бізнес-початок місяця           |

---

### Квартали

| Частота       | Alias              | Опис                                    |
| ------------- | ------------------ | --------------------------------------- |
| QuarterEnd    | `'Q'` або `'QE'`   | кінець календарного кварталу            |
| QuarterBegin  | `'QS'`             | початок кварталу                        |
| BQuarterEnd   | `'BQ'` або `'BQE'` | бізнес-кінець кварталу                  |
| BQuarterBegin | `'BQS'`            | бізнес-початок кварталу                 |

**Приклади**

* `'Q-MAR'` — квартал, що закінчується у березні
* `'QS-APR'` — квартал починається у квітні

---

### Річні частоти

| Частота    | Alias                   | Опис                       |
| ---------- | ----------------------- | -------------------------- |
| YearEnd    | `'A'` або `'YE'`        | кінець календарного року   |
| YearBegin  | `'AS'`, `'YS'`, `'BYS'` | початок року               |
| BYearEnd   | `'BA'`, `'BYE'`         | бізнес-кінець року         |
| BYearBegin | `'BAS'`, `'BYS'`        | бізнес-початок року        |


**Приклади**

* `'A-DEC'` — рік закінчується у грудні
* `'AS-JUL'` — рік починається у липні

In [ ]:
# Підготовка даних


In [ ]:
# Приклад: середня ціна квартир по місяцях


In [22]:
# Візуалізація: динаміка середньої ціни по місяцях


In [21]:
# Кількість оголошень по тижнях


In [20]:
# Візуалізація: кількість оголошень по тижнях


## Навіщо аналітику `resample`

| Що дізнаємось                         | Як це корисно                       |
| ------------------------------------- | ----------------------------------- |
| Сезонність ринку                      | порівняння цін по місяцях/роках     |
| Активність користувачів               | по кількості оголошень              |
| Аналіз впливу подій (інфляція, війна) | тренди по кварталах                 |
| Маркетинг                             | коли зростає інтерес до нерухомості |

---

## Нюанси

| Проблема          | Рішення                                 |
| ----------------- | --------------------------------------- |
| Не дата в індексі | `.set_index('datetime')`                |
| Пропущені періоди | Автоматично створюються `NaN`           |
| Смугастий графік  | Використовуйте `'M'`, `'Q'`, а не `'D'` |
| Великі значення   | Обрізати викиди перед побудовою         |

---

# `pd.cut()` — групування числових значень у категорії

## Що таке `cut`

`cut()` розбиває числову колонку на **інтервали (бінінг)**. Наприклад, можна створити категорії квартир:

* до 40 м² → «мала»
* 40–80 м² → «середня»
* понад 80 м² → «велика»

Це корисно для:

* **аналізу розподілу** (гістограми, pie charts)
* **порівнянь середніх цін між групами**
* **створення сегментів**

---

## Синтаксис

```python
pd.cut(Series, bins, labels=..., include_lowest=True)
```

| Параметр         | Значення                         |
| ---------------- | -------------------------------- |
| `bins`           | кількість або список меж         |
| `labels`         | імена категорій                  |
| `include_lowest` | включити нижню межу в перший бін |

---

In [19]:
# Приклад: Розбиття квартир за площею


In [18]:
# Візуалізація: Середня ціна за розміром квартири


In [17]:
# Приклад: Розбиття цін на групи



# Аналіз середньої площі в кожній ціновій категорії




In [16]:
# Альтернатива: qcut() — автоматичне розбиття на квантилі
# Корисно, коли хочемо однакову кількість спостережень у групі, а не однакову ширину інтервалу.


In [15]:
# Візуалізація: розподіл площі по цінових квантилях


## Навіщо аналітику `cut` і `qcut`

| Задача                             | Рішення                |
| ---------------------------------- | ---------------------- |
| Побудова сегментів                 | `cut` або `qcut`       |
| Когортний аналіз (покоління, ціна) | `cut` по даті або ціні |
| Спрощення числових фіч             | категоризація          |
| Візуалізація (pie, bar, boxplot)   | категорії зручніші     |

---

## Типові помилки

| Проблема                          | Рішення                                   |
| --------------------------------- | ----------------------------------------- |
| Значення на межі не включене      | `include_lowest=True`                     |
| Некоректні категорії              | задати `labels` вручну або перевірити     |
| Багато унікальних значень у `cut` | замість `cut` — використовувати `qcut`    |
| Нерівномірні категорії            | вручну задати `bins`, якщо треба контроль |

---